In [1]:
from loguru import logger
import math
import subprocess
from pathlib import Path

import yaml

with open("../run.yaml", "r") as f:
    config_template = yaml.safe_load(f)

In [2]:
def write_slurm_script(work_dir: str | Path, config: dict, script_content: str) -> Path:
    """
    Generate a SLURM job script based on the provided configuration.

    Args:
        config (dict): Parsed YAML configuration.
        work_dir (str): Directory to write the SLURM job script.
        script_content (str): Bash commands to run in the SLURM job script.
    """

    work_dir = Path(work_dir)
    work_dir.mkdir(parents=True, exist_ok=True)

    with open(work_dir / "run.yaml", "w") as f:
        yaml.dump(config, f, sort_keys=False)

    scheduler = config["scheduler"]
    resources = scheduler["resources"]
    pre_run = scheduler.get("pre_run", "")
    post_run = scheduler.get("post_run", "")
    modules = scheduler.get("modules", []) or []

    # Define the SLURM script path
    script_path = work_dir / "job.slurm"

    # Write the SLURM script
    with open(script_path, "w") as f:
        f.write(f"""#!/bin/bash
#SBATCH --job-name={resources["job_name"]}
#SBATCH --output={resources["output"]}
#SBATCH --error={resources["error"]}
#SBATCH --time={resources["time"]}
#SBATCH --partition={resources["partition"]}
#SBATCH --nodes={resources["nodes"]}
#SBATCH --ntasks-per-node={resources["ntasks_per_node"]}

# Pre-run commands
{pre_run if pre_run else ""}

# Load required modules
""")

        for module in modules:
            f.write(f"module load {module}\n")

        f.write(f"""
{script_content}
""")

        f.write(f"\n# Post-run commands\n{post_run if post_run else ''}\n")

    logger.info(f"SLURM job script written to: {script_path}")
    return script_path

In [3]:
import itertools

ncpus_per_node = 64

NNODEs = [1, 2, 4, 8, 16]
# NCOREs = [1, 2, 4, 8, 16, 32, 64]
NCOREs = [2**i for i in range(int(math.log2(ncpus_per_node)) + 1)]
# KPARs = [1, 2, 4, 8, 16]  # <= KPOINTS = 2x2x4 = 16 in this case
KPARs = [2**i for i in range(int(math.log2(16)) + 1)]

for comb in itertools.product(NNODEs, NCOREs, KPARs):
    NNODE, NCORE, KPAR = comb
    NPAR = ncpus_per_node / KPAR / NCORE

    if not NPAR.is_integer():
        continue

    logger.info(f"NNODE: {NNODE}, NCORE: {NCORE}, KPAR: {KPAR}, NPAR: {NPAR}")
    logger.info(f"Total ranks (NNODE x NCORE x KPAR): {NNODE * NCORE * KPAR}")

    config = config_template.copy()

    config["job"]["vasp"]["INCAR"].update(
        {
            "NCORE": NCORE,
            "KPAR": KPAR,
            # "NPAR": int(NPAR),
        }
    )

    config["scheduler"]["resources"].update(
        {
            "nodes": NNODE,
            "ntasks_per_node": ncpus_per_node,
        }
    )

    work_dir = Path(f"{NNODE}-{NCORE}-{KPAR}")

    script_path = write_slurm_script(
        work_dir=work_dir,
        config=config,
        script_content="python ../workflow.py run.yaml",
    )

    subprocess.run(["sbatch", script_path.name], cwd=script_path.absolute().parent)
    logger.info(f"Submitted job {script_path}")
        

2025-01-17 23:26:01.594 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 1, KPAR: 1, NPAR: 64.0
2025-01-17 23:26:01.595 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 1
2025-01-17 23:26:01.599 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-1-1/job.slurm
2025-01-17 23:26:02.047 | INFO     | __main__:<module>:47 - Submitted job 1-1-1/job.slurm
2025-01-17 23:26:02.048 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 1, KPAR: 2, NPAR: 32.0
2025-01-17 23:26:02.049 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 2
2025-01-17 23:26:02.052 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-1-2/job.slurm
2025-01-17 23:26:02.097 | INFO     | __main__:<module>:47 - Submitted job 1-1-2/job.slurm
2025-01-17 23:26:02.098 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 1, KPAR: 4, NPAR: 16.0
2025-01-17 23:26:02.098 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR)

Submitted batch job 28435407
Submitted batch job 28435408
Submitted batch job 28435409
Submitted batch job 28435410
Submitted batch job 28435411
Submitted batch job 28435412
Submitted batch job 28435413
Submitted batch job 28435414
Submitted batch job 28435415


2025-01-17 23:26:02.262 | INFO     | __main__:<module>:47 - Submitted job 1-2-16/job.slurm
2025-01-17 23:26:02.262 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 4, KPAR: 1, NPAR: 16.0
2025-01-17 23:26:02.263 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 4
2025-01-17 23:26:02.267 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-4-1/job.slurm
2025-01-17 23:26:02.282 | INFO     | __main__:<module>:47 - Submitted job 1-4-1/job.slurm
2025-01-17 23:26:02.282 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 4, KPAR: 2, NPAR: 8.0
2025-01-17 23:26:02.283 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 8
2025-01-17 23:26:02.287 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-4-2/job.slurm
2025-01-17 23:26:02.303 | INFO     | __main__:<module>:47 - Submitted job 1-4-2/job.slurm
2025-01-17 23:26:02.304 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 4, KPAR: 4, NPAR: 4.0
2025-

Submitted batch job 28435416
Submitted batch job 28435417
Submitted batch job 28435418
Submitted batch job 28435419
Submitted batch job 28435420
Submitted batch job 28435421
Submitted batch job 28435422
Submitted batch job 28435423


2025-01-17 23:26:03.712 | INFO     | __main__:<module>:47 - Submitted job 1-8-4/job.slurm
2025-01-17 23:26:03.712 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 8, KPAR: 8, NPAR: 1.0
2025-01-17 23:26:03.713 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 64
2025-01-17 23:26:03.716 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-8-8/job.slurm
2025-01-17 23:26:03.749 | INFO     | __main__:<module>:47 - Submitted job 1-8-8/job.slurm
2025-01-17 23:26:03.749 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 16, KPAR: 1, NPAR: 4.0
2025-01-17 23:26:03.750 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 16
2025-01-17 23:26:03.754 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 1-16-1/job.slurm
2025-01-17 23:26:03.769 | INFO     | __main__:<module>:47 - Submitted job 1-16-1/job.slurm
2025-01-17 23:26:03.769 | INFO     | __main__:<module>:18 - NNODE: 1, NCORE: 16, KPAR: 2, NPAR: 2.0
2

...

Submitted batch job 28435538
Submitted batch job 28435539


2025-01-17 23:26:13.097 | INFO     | __main__:<module>:47 - Submitted job 16-2-16/job.slurm
2025-01-17 23:26:13.098 | INFO     | __main__:<module>:18 - NNODE: 16, NCORE: 4, KPAR: 1, NPAR: 16.0
2025-01-17 23:26:13.098 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 64
2025-01-17 23:26:13.103 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 16-4-1/job.slurm
2025-01-17 23:26:13.136 | INFO     | __main__:<module>:47 - Submitted job 16-4-1/job.slurm
2025-01-17 23:26:13.136 | INFO     | __main__:<module>:18 - NNODE: 16, NCORE: 4, KPAR: 2, NPAR: 8.0
2025-01-17 23:26:13.137 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 128
2025-01-17 23:26:13.142 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 16-4-2/job.slurm
2025-01-17 23:26:13.161 | INFO     | __main__:<module>:47 - Submitted job 16-4-2/job.slurm
2025-01-17 23:26:13.162 | INFO     | __main__:<module>:18 - NNODE: 16, NCORE: 4, KPAR: 4, NPAR

Submitted batch job 28435540
Submitted batch job 28435541
Submitted batch job 28435542
Submitted batch job 28435543
Submitted batch job 28435544
Submitted batch job 28435545
Submitted batch job 28435546
Submitted batch job 28435547
Submitted batch job 28435548


2025-01-17 23:26:13.310 | INFO     | __main__:<module>:47 - Submitted job 16-8-8/job.slurm
2025-01-17 23:26:13.311 | INFO     | __main__:<module>:18 - NNODE: 16, NCORE: 16, KPAR: 1, NPAR: 4.0
2025-01-17 23:26:13.311 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 256
2025-01-17 23:26:13.315 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 16-16-1/job.slurm
2025-01-17 23:26:13.333 | INFO     | __main__:<module>:47 - Submitted job 16-16-1/job.slurm
2025-01-17 23:26:13.334 | INFO     | __main__:<module>:18 - NNODE: 16, NCORE: 16, KPAR: 2, NPAR: 2.0
2025-01-17 23:26:13.334 | INFO     | __main__:<module>:19 - Total ranks (NNODE x NCORE x KPAR): 512
2025-01-17 23:26:13.338 | INFO     | __main__:write_slurm_script:52 - SLURM job script written to: 16-16-2/job.slurm
2025-01-17 23:26:13.425 | INFO     | __main__:<module>:47 - Submitted job 16-16-2/job.slurm
2025-01-17 23:26:13.426 | INFO     | __main__:<module>:18 - NNODE: 16, NCORE: 16, KPAR: 4

Submitted batch job 28435549
Submitted batch job 28435550
Submitted batch job 28435551
Submitted batch job 28435552
Submitted batch job 28435553
Submitted batch job 28435554
Submitted batch job 28435555


In [27]:
from mp_api.client import MPRester

with MPRester() as mpr:
    docs = mpr.materials.summary.search(
        chemsys=["Pb-Ti-O", "Pb-Zr-O", "Pb-Ti-Zr-O"],
        spacegroup_symbol="P4mm",  # "Pm-3m", "P4mm" "R3mm"
        fields=["material_id", "symmetry", "structure"],
    )

structure = docs[0].structure

Retrieving SummaryDoc documents: 100%|██████████| 3/3 [00:00<00:00, 52428.80it/s]


In [29]:
with MPRester() as mpr:
    docs = mpr.materials.summary.search(
        chemsys=["Pb-Ti-O", "Pb-Zr-O", "Pb-Ti-Zr-O"],
        spacegroup_symbol="P4mm",  # "Pm-3m", "P4mm" "R3mm"
        fields=["material_id", "symmetry", "structure"],
    )

structure = docs[0].structure

structure = docs[1].structure * (3, 3, 2)
structure.to(filename=f"POSCAR.{structure.to_ase_atoms().get_chemical_formula()}")

Retrieving SummaryDoc documents: 100%|██████████| 3/3 [00:00<00:00, 79137.81it/s]
/ocean/projects/dmr110014p/ychiang4/.conda/envs/comm/lib/python3.12/site-packages/pymatgen/io/vasp/inputs.py:660: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as file:


'Zr18 Ti18 Pb36 O108\n1.0\n  12.0827400000000011    0.0000000000000000    0.0000000000000000\n   0.0000000000000000   12.0827400000000011    0.0000000000000000\n   0.0000000000000000    0.0000000000000000   17.5231080000000006\nZr Ti Pb O\n18 18 36 108\ndirect\n   0.1666666666666667    0.1666666666666667    0.3819460000000000 Zr\n   0.1666666666666667    0.1666666666666667    0.8819460000000001 Zr\n   0.1666666666666667    0.5000000000000000    0.3819460000000000 Zr\n   0.1666666666666667    0.5000000000000000    0.8819460000000001 Zr\n   0.1666666666666667    0.8333333333333333    0.3819460000000000 Zr\n   0.1666666666666667    0.8333333333333333    0.8819460000000001 Zr\n   0.5000000000000000    0.1666666666666667    0.3819460000000000 Zr\n   0.5000000000000000    0.1666666666666667    0.8819460000000001 Zr\n   0.5000000000000000    0.5000000000000000    0.3819460000000000 Zr\n   0.5000000000000000    0.5000000000000000    0.8819460000000001 Zr\n   0.5000000000000000    0.83333333333